data_processing.ipynb
by: 
- Sheil Kumar (sk17@illinois.edu)
- Anirudh Eswara (aeswara2@illinois.edu)
- Lloyd Fernandes (lloydf2@illinois.edu)

Project director: Richard Sowers
r-sowers@illinois.eduhttps://publish.illinois.edu/r-sowers/
Copyright 2019 University of Illinois Board of Trustees. All Rights Reserved. Licensed under the MIT license


# ETL on pNEUMA dataset
This notebook serves to show how the features in `data.py` and `feature_eng.py` is used to clean and augment the data downloaded from 



In [ ]:
from data_loader import *
from feature_eng import *

## 1.0 Data.py

- Functions:
  - `csv_to_df`
  - `pickle_df`

In [2]:
#The original dataset
pd.read_csv("block4_1_concat.csv",nrows = 5)

,track_id; type; traveled_d; avg_speed; lat; lon; speed; lon_acc; lat_acc; time
0,1; Taxi; 182.37; 9.740748; 37.982746; 23.73296...
1,2; Taxi; 305.97; 24.261816; 37.982849; 23.7331...
2,3; Taxi; 82.72; 12.947546; 37.981815; 23.73386...
3,4; Taxi; 177.45; 13.534007; 37.981281; 23.7339...
4,5; Taxi; 497.24; 35.237767; 37.984355; 23.7320...


In [27]:
#csv_to_df converts the above displayed csv to a multi Index dataframe
df = csv_to_df('block4_d4_3.csv') #simply need to feed in name of .csv file containing dat a11
df.to_pickle('block4_d4_3.pkl') #simply need to feed in dataframe and desired name for .pkl file 
df.head()

lat        lon    speed  lon_acc  lat_acc type  traveled_d  \
id time                                                                     
1  4.20  37.982487  23.732436  20.5231   0.3241   0.0016  Car      281.22   
   4.24  37.982489  23.732438  20.5505   0.3241   0.0017  Car      281.22   
   4.28  37.982490  23.732440  20.5774   0.3252   0.0018  Car      281.22   
   4.32  37.982491  23.732442  20.6041   0.3270   0.0019  Car      281.22   
   4.36  37.982492  23.732444  20.6307   0.3273   0.0021  Car      281.22   

         avg_speed  
id time             
1  4.20   7.510226  
   4.24   7.510226  
   4.28   7.510226  
   4.32   7.510226  
   4.36   7.510226

## 2.0 feature_eng.py

- Functions:
    - `bearing`
    - `nearest_graph_data`
    - `direction`
    - `vehicle_density` 
    - `cross_track`
    - `edge_average_speed`
    - `split_trajectories`

In [28]:
#Add bearings
df = bearing(df) #using bearing will then return a df with bearing as a column 
df.head()

lat        lon    speed  lon_acc  lat_acc type  traveled_d  \
id time                                                                     
1  4.20  37.982487  23.732436  20.5231   0.3241   0.0016  Car      281.22   
   4.24  37.982489  23.732438  20.5505   0.3241   0.0017  Car      281.22   
   4.28  37.982490  23.732440  20.5774   0.3252   0.0018  Car      281.22   
   4.32  37.982491  23.732442  20.6041   0.3270   0.0019  Car      281.22   
   4.36  37.982492  23.732444  20.6307   0.3273   0.0021  Car      281.22   

         avg_speed   bearing  
id time                       
1  4.20   7.510226  0.168068  
   4.24   7.510226  0.327147  
   4.28   7.510226  0.327151  
   4.32   7.510226  0.327154  
   4.36   7.510226  0.470846

In [8]:
#add nearest nodes and edges
df = pd.read_pickle("block4_1_3_feat.pkl")
lat = df.iloc[1,0]
lon = df.iloc[1,1]
graph = ox.graph_from_point((lat,lon), network_type='drive', dist=700)
#graph = ox.graph_from_address('Athens, Municipality of Athens, Regional Unit of Central Athens, Attica, 10667, Greece', network_type='drive')  #need to provide graph from osmnx as well 
df = nearest_graph_data(df, graph,mode='balltree') #nearest_graph_data will then return multiple features regarding the node and edge per id 
df.to_pickle("block4_1_3_feat.pkl")

In [3]:
df.head()

lat        lon    speed  lon_acc  lat_acc type  \
id traj time                                                         
26 0    0.00  37.982707  23.733368  27.7221  -0.0244   0.2467  Car   
        0.04  37.982705  23.733370  27.7208   0.0058   0.2835  Car   
        0.08  37.982703  23.733373  27.7229   0.0244   0.3199  Car   
        0.12  37.982701  23.733375  27.7278   0.0431   0.3557  Car   
        0.16  37.982698  23.733377  27.7366   0.0791   0.3908  Car   

              traveled_d  avg_speed   bearing  edge_progress  dir  \
id traj time                                                        
26 0    0.00      268.16    11.9479  2.972632       0.412546    1   
        0.04      268.16    11.9479  2.891083       0.417071    1   
        0.08      268.16    11.9479  2.972627       0.422278    1   
        0.12      268.16    11.9479  3.028350       0.426808    1   
        0.16      268.16    11.9479  2.972624       0.432919    1   

              edge_progress_intervals  vehicle_density  avg_surr_speed  \
id traj time                                                             
26 0    0.00                      0.4                1         27.7221   
        0.04                      0.4                1         27.7208   
        0.08                      0.4                1         27.7229   
        0.12                      0.4                1         27.7278   
        0.16                      0.4                1         27.7366   

              xtrack_dist  nearest_edge_start_node  nearest_edge_end_node  
id traj time                                                               
26 0    0.00     2.108172                250699362              599116897  
        0.04     2.081480                250699362              599116897  
        0.08     1.981148                250699362              599116897  
        0.12     1.954456                250699362              599116897  
        0.16     1.988058                250699362              599116897

In [ ]:
#find edges which are greater than 65m in length


#remove trajectories where edge is smaller than 65m
df = df[df.edge_id.isin(edge_lane.edge_id)]

In [ ]:
# add vehicle direction wrt edge
df = direction(df) # running direction on dataframe will then return a df with the direction as a column 
df.head()

In [ ]:
#Add Cross track distance
df = cross_track(df,graph)
df.head()

In [1]:
#assign edge_id    
df["edge_id"] = df["nearest_edge_start_node"].astype(str)+"_"+df["nearest_edge_end_node"].astype(str)

df_nodes = df.reset_index()[['nearest_edge_start_node','nearest_edge_end_node','edge_id']].drop_duplicates()
#find length of each edge
df_nodes['len'] = df_nodes.apply(lambda x: length(x[0],x[1],graph),axis = 1)

df['len']  = df['edge_id'].map(df_nodes.set_index('edge_id')['len'])
df['lanes']  = df['edge_id'].map(edge_lane.set_index('edge_id')['lanes'])

#remove trajectory outside the lane, save vehicles which for some duration move outside of the lane in df_out_of_lane dataframe
df,df_out_of_lane = remove_traj_outside_lane(df,xtrack_lim = 0.75)

#remove trajectory near 10meters of intersection
df = remove_traj_near_nodes(df,node_rad = 10)
#add a column defining which segment of the edge the vehicle lies in 
df = split_edge_to_seg(df,seg_len = 20,seg_lim = 17)
# find number of vehicles in segment where the subject vehicle is located
df = vehicle_density_by_seg(df)
#find average surrounding vehicle speed where the subject vehicle is located
df = avg_surr_speed_by_seg(df)

#save as pickle file
df.to_pickle('block4_'+file_name+'_filt_lane.pkl')
    
#remove all vehicles which are in df_out_of_lane list
df = df[ ~(df.index.get_level_values('id').isin(df_out_of_lane.id.to_list()))]

#add filename in index and save
df = df[df.type.isin(['Car','Taxi'])]
df['file_name'] = file_name
df.set_index('file_name', append=True, inplace=True)
df.reorder_levels([2,0,1])
df.to_pickle('block4_'+file_name+'_ct_lane.pkl')

    

IndentationError: unexpected indent (<ipython-input-1-0e3908568911>, line 25)